[기초부터 시작하는 NLP: 문자-단위 RNN으로 이름 분류하기](https://tutorials.pytorch.kr/intermediate/char_rnn_classification_tutorial.html)

## 문자 단위 RNN이름 분류

__단어를 분류하기 위해 기초적인 문자 단위 RNN 학습__\
NLP 모델링을 통해 torchtext의 편리한 기능을 사용하지 않고 데이터를 처리하는 방법 익힘\
단어를 문자의 연속으로 읽어서 각 단계의 예측과 은닉 상태 출력, 다음 단계에 이전 단계의 은닉 상태 전달\
단어가 속한 클래스로 출력되도록 최종 예측 선택

예) 철자에 따라 이름이 어떤 언어인지 예측하기
```bash
$ python predict.py Hinton
(-0.47) Scottish
(-1.52) English
(-3.57) Irish

$ python predict.py Schmidhuber
(-0.19) German
(-2.48) Czech
(-2.68) Dutch
```